# Fingerprints to SMILES amb MolForge (executar en l'entorn MolForge-env)

1. Llegeix el fitxer de fingerprints preprocessat amb columnes: "id" y "fingerprints_input".  
2. Executa MolForge sobre els fingerprints y guarda els resultats en una nova columna "SMILES_output".  
3. Guarda els resultats en `data/MolForge_output/`.

## Imports

In [1]:
# Per definir l'arrel del projecte
import os

# Pandas pels dataframes
import pandas as pd
import numpy as np

# Per cridar MolForge i guardar-ne l'output
from MolForge import main as molforge_main
import sys
import io
from contextlib import redirect_stdout

## Inputs (part a editar)

Arrel del projecte

In [2]:
os.chdir("/export/home/ddiestre/MolForge_Testing")

Fitxer de fingerprints preprocessat (path a partir de MolForge_Testing/)

In [3]:
input_path = "data/MolForge_input/test_1.csv"

Fitxer en que guardar l'output (path a partir de MolForge_Testing/)

In [4]:
output_path = "data/MolForge_output/test_1_output.csv"

Paràmetres de MolForge

In [5]:
FP_NAME = "ECFP4"
MODEL_TYPE = "smiles"  # ["smiles", "selfies"]
DECODE = "greedy"  # ["greedy", "beam"]
CHECKPOINT_NAME = "ECFP4_smiles_checkpoint.pth"

## 1. Lectura del fitxer

In [6]:
# Lectura del fitxer
df = pd.read_csv(input_path, sep = ',', index_col = 0)
df.head(6)

,fingerprints_input
id,
1,1 80 94 114 237 241 255 294 392 411 425 695 74...
2,97 101 314 378 389 442 501 650 728 817 896 909...
3,9 45 78 89 145 203 322 548 586 650 695 718 760...
4,bfeifjer
5,InvalidSMILE
6,NaN


## 2. Execució de Molforge

In [7]:
def run_for_fp(input_fp, fp_name, model_type, checkpoint_name, decode):
    # 0. Tractem els casos "NaN" i "InvalidSMILE"
    if (input_fp == "InvalidSMILE") or (input_fp is np.nan):
        return np.nan

    # 1. Guardar sys.argv original (per restaurar-lo després)
    original_argv = sys.argv
    
    # 2. Crear els arguments com si s'haguessin passat des de CLI (Command Line Interface)
    sys.argv = [
        "",  # el primer argument de sys.argv s'ignora
        f"--input={input_fp}",
        f"--fp={fp_name}",
        f"--model_type={model_type}",
        f"--checkpoint={checkpoint_name}",
        f"--decode={decode}",
    ]
    
    # 3. Capturar stdout
    buf = io.StringIO()  # creem un buffer text en memòria (on guardarem l'output de MolForge)
    try:  # per asegurar-nos que restaurem sys.argv encara que peti
        with redirect_stdout(buf):  # guardem l'output de molforge a buf
            molforge_main()  # executem molforge_main amb els arguments de sys.argv
    finally:
        # 4. Restaurar sys.argv (important en notebooks! tot i que no és estrictament necessari en aquest cas, és una bona pràctica)
        sys.argv = original_argv
    
    # 5. Processar resultat
    output = buf.getvalue()  # transforma buf en string
    for line in output.splitlines():
        line = line.strip()  # elimina espais i salts de línia al principi i al final
        if line.startswith("Result:"):
            return line.split("Result:", 1)[1].strip().replace(" ", "")  # separem per "Result:" i agafem només la part de després fent-li un strip i treient-li espais
    
    return None

In [8]:
smiles_out = [] # aquesta serà la nostra columna SMILES_output
len_df = len(df)
for i in range(len_df): # de 0 a len_df-1
    s = run_for_fp(df['fingerprints_input'][i+1], FP_NAME, MODEL_TYPE, CHECKPOINT_NAME, DECODE)
    smiles_out.append(s)
    print(f"\r[{i+1}/{len_df}]", end="", flush=True) # Seguiment del progrés

[6/6]

## 3. Guardar l'output

In [9]:
# Creem el nou dataframe
df['SMILES_output'] = smiles_out
df.head(6)

,fingerprints_input,SMILES_output
id,,
1,1 80 94 114 237 241 255 294 392 411 425 695 74...,CCOC1=C(C=C(C=C1)C(C(C)(C)C)N)OCC
2,97 101 314 378 389 442 501 650 728 817 896 909...,C1=CC=C(C=C1)C2C=C(NC(=O)C2C3=NC(=S)NC(=O)C34C...
3,9 45 78 89 145 203 322 548 586 650 695 718 760...,COC1=CC=C(C=C1)C(=O)C2=C(C(=C3N2C4=CC=CC=C4C=C...
4,bfeifjer,C12=C3C4=C5C1=C6C27C38C49C1=C2C8=C9C1=C1C2=C1C...
5,InvalidSMILE,NaN
6,NaN,NaN


In [10]:
df.to_csv(output_path)